In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [52]:
import base64
import json
import logging
import os
import re
import smtplib
import sys
import time
import threading
from copy import deepcopy
from functools import partial
from itertools import chain, combinations, product, permutations
from numbers import Number
from pathlib import Path
from uuid import uuid4
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser
from scipy.optimize import minimize
from scipy.special import comb, perm

from api.risk.var import (
    VaRPnl,
    get_db_from_clarion_positions, 
    get_fut_trade_spec, get_var_ts, 
    get_db_from_specs, var_ptf, 
    get_clarion_fut_trade_spec, 
    cvar_ptf, bbg_stir_to_clarion, 
    get_positions_in_book
)
from api.email import smtp_server, to_email_test, to_email_all, df_to_html, from_email
from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods, parse_swap_period
from gioutils.tools_clarion import get_clarion_positions
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
    SQL
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_specific_future,
    get_bbg_fut_chain_ticker,
    get_root,
    encode_df_as_image_tag
)

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)
rng = np.random.default_rng(7)

In [4]:
super_clarion(clarion)
runner = Runner(clarion)

In [5]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-05-16 08:25:19,398 - Authenticating...
2023-05-16 08:25:20,051 - Successfully authenticated


In [6]:
def send_live_var_email(
    var_close, 
    var_intraday, 
    var_new, 
    intraday_group, 
    trades_show,
    subject=f'Live VaR Reduction {pd.Timestamp.now(): %d%b%y %H:%M}'
):
    
    to_email = [
        'giovambattista.perciaccante@brevanhoward.com',
        'menashe.banit@brevanhoward.com'
    ]
    
    font_family = 'Helvetica, Arial, Sans-Serif'
    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = from_email
    message["To"] = ','.join(to_email)

    file_name = 'tempVaR'
    dfs = [encode_df_as_image_tag(s, table_conversion='m') for s in [intraday_group, trades_show]]
    html_body = f"<h3 style=\"font-family:{font_family};\">Close VaR:{'&nbsp;'*40}{var_close: .2%}</h3>"
    html_body = html_body + f"<h3 style=\"font-family:{font_family};\">VaR with Intraday (below): {'&nbsp;'*13}{var_intraday: .2%}</h3>"
    html_body = html_body + dfs[0] + '<br>' * 3
    html_body = html_body + f"<h3 style=\"font-family:{font_family};\">VaR after the below trades:{'&nbsp;'*11}{var_new: .2%}</h3>"
    html_body = html_body + dfs[1] 
    html_ = MIMEText(html_body, "html")
    message.attach(html_)
    smtp = smtplib.SMTP(smtp_server)
    # smtp.set_debuglevel(1)
    smtp.sendmail(from_email, to_email, message.as_string())

In [7]:
def send_live_var_email(var_close, var_intraday, var_new, intraday_group, trades_show, test=False):
    to_email = to_email_all if not test else to_email_test
    font_family = 'Helvetica, Arial, Sans-Serif'

    message = MIMEMultipart("alternative")
    message["Subject"] = f'LIVE VaR REDUCTION - {pd.Timestamp.now(): %d%b%y %H:%M}'
    message["From"] = from_email
    message["To"] = ','.join(to_email)

    # dfs = [encode_df_as_image_tag(s) for s in [intraday_group, trades_show]]

    dfs = [df_to_html(s.style.format(millify)) for s in [intraday_group, trades_show]]
    html_body = f"<h3 style=\"font-family:{font_family};\">Close VaR:{'&nbsp;' * 40}{var_close: .2%}</h3>"
    html_body = html_body + f"<h3 style=\"font-family:{font_family};\">VaR with Intraday (below): {'&nbsp;' * 13}{var_intraday: .2%}</h3>"
    html_body = html_body + dfs[0] + '<br>' * 3
    html_body = html_body + f"<h3 style=\"font-family:{font_family};\">VaR after the below trades:{'&nbsp;' * 11}{var_new: .2%}</h3>"
    html_body = html_body + dfs[1]
    html_ = MIMEText(html_body, "html")
    message.attach(html_)
    smtp = smtplib.SMTP(smtp_server)
    # smtp.set_debuglevel(1)
    smtp.sendmail(from_email, to_email, message.as_string())


### METRICS

In [8]:
clarion_metrics = dict(
    pv = clarion.metric('pv', 'PresentValue', 'Any'),
    local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any'),
    # currency = clarion.metric('currency', 'PayCurrency', 'Any'),
    strike = clarion.metric('strike', 'Strike', 'Any'),
    notional = clarion.metric('notional', 'Notional', 'Any'),
    expiry = clarion.metric('expiry', 'ExpiryDate', 'Any'),
    maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'}),
    start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'}),
    fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates'),
    delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    gamma = clarion.metric('gamma', 'Gamma', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'}),
    price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'}),
    fwd_price_cents = clarion.metric('fwd_price_cents', 'ForwardPrice', 'Rates', {'Unit': 'Cents'}),
    price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'}),
    price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'}),
    price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'}),
    spot = clarion.metric('spot', 'Spot', 'Rates'),
    # vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'}),
    # atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'}),
    atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'}),
    # strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'}),
    strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'}),
    # carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'}),
)

### VAR

In [9]:
capital = 300e6
flag = 1.5 / 100 * capital
book = 'MM'

In [10]:
dv = VaRPnl()
wv = VaRPnl()

In [75]:
dv.tail(capital=capital)

returnDate
2023-03-16   -3.2214
dtype: float64

In [70]:
dv.tail(get_positions_in_book({'FFX EUR JPY 20230608 150.062': -15e6}, dv.ref, dv.hist), capital=capital, add=True)

returnDate
2023-03-16   -3.1697
dtype: float64

In [66]:
wv.tail(capital=capital)

returnDate
2022-09-23   -2.4804
dtype: float64

In [74]:
wv.tail(get_positions_in_book({'FFX EUR JPY 20230608 150.062': -15e6}, wv.ref, wv.hist), capital=capital, add=True)

returnDate
2022-09-23   -2.3185
dtype: float64

In [51]:
dv.ref.query("strategy=='MM:FX OVERLAY'")

0  fund book       strategy                  positionName riskCategory  \
62  BAL   MM  MM:FX OVERLAY                      CASH EUR           FX   
63  BAL   MM  MM:FX OVERLAY                      CASH GBP           FX   
64  BAL   MM  MM:FX OVERLAY                      CASH ILS           FX   
65  BAL   MM  MM:FX OVERLAY                      CASH JPY           FX   
66  BAL   MM  MM:FX OVERLAY                      CASH NOK           FX   
67  BAL   MM  MM:FX OVERLAY  FFX EUR JPY 20230608 150.062           FX   

0  currency tradeGroup instrument underlying_name   isAO  \
62      EUR       CASH        EUR            NULL  False   
63      GBP       CASH        GBP            NULL  False   
64      ILS       CASH        ILS            NULL  False   
65      JPY       CASH        JPY            NULL  False   
66      NOK       CASH        NOK            NULL  False   
67      EUR        FXD    EUR/JPY            NULL  False   

0              positionName2       minnb  nominal   delta  
62                  CASH EUR -2014616653  228.95k       0  
63                  CASH GBP -2014617984   7.8100       0  
64                  CASH ILS -2014620219  -33.95k       0  
65                  CASH JPY -2014621310   43.86m       0  
66                  CASH NOK -2014625109 -319.28k       0  
67  FFX JPY 20230608 150.062    31074223     -30m -33.08m

In [11]:
dv.get_data()
wv.get_data(
    offset='5y',
    horizon='5d',
    book=book
)
f"VaR: {- dv.var() / flag:.2%}"

C:\dev\projects\menashe\api\risk\var.py:172: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ).sum()
C:\dev\projects\menashe\api\risk\var.py:172: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ).sum()


'VaR: 34.90%'

In [46]:
cob_ptf = dv.hist.sum(axis=1).rename('ptf')
cob_ptf_w = wv.hist.sum(axis=1).rename('ptf')
# cob_ptf = dv.hist \
#     .drop([*dv.ref.query('tradeGroup == "OPT" and instrument == "USD CBT T 10Y"')['positionName']] , axis=1, level=1) \
#     .drop('MM:BOE FEB MAY', axis=1) \
#     .drop(
#         [
#             'USD SOFR CME 3M P 94.812 20230310',
#             'USD SOFR CME 3M P 94.938 20230310',
#         ]
#     , axis=1, level=1) \
#     .sum(axis=1).rename('ptf')

In [13]:
f"Cob VaR: {- cob_ptf.quantile(0.05, interpolation='weibull') / flag:.2%}"

'Cob VaR: 34.87%'

### ADD INTRADAY POSITIONS

In [14]:
df = get_clarion_positions(clarion)
clear_output()
intraday = df.query('isIntraday == True')
intraday_group = intraday.groupby('Description')['Notional'].sum().replace(0, np.nan).dropna().to_frame()
rates_mask = intraday.loc[intraday['Category'] == 'rates', 'Description'].drop_duplicates().to_list()
intraday_rates = intraday_group.loc[intraday_group.index.isin(rates_mask)]
intraday_group

Empty DataFrame
Columns: [Notional]
Index: []

### LIVE VAR

In [47]:
if not intraday_rates.empty:
    db = get_db_from_clarion_positions(intraday)
    intraday_trades, res = get_var_ts(db)
    current_ptf = pd.concat([cob_ptf, intraday_trades], axis=1).dropna().sum(axis=1)
    current_ptf_w = pd.concat([cob_ptf_w, intraday_trades.resample('W-FRI').sum()], axis=1).dropna().sum(axis=1)
else:
    current_ptf = cob_ptf
    current_ptf_w = cob_ptf_w

In [48]:
display(f"Live VaR: {- current_ptf.quantile(0.05, interpolation='weibull') / flag:.2%}")
display(f"Live Daily Tail: {dv.tail(current_ptf, capital=capital).round(2).squeeze()}")
display(f"Live Weekly Tail: {wv.tail(current_ptf_w, capital=capital).round(2).squeeze()}")

'Live VaR: 34.87%'

'Live Daily Tail: -3.22'

'Live Weekly Tail: -2.48'

### CUSTOM ADDITIONS

In [ ]:
add_trades_list = [
    ('sfr', 'h24', 500),
    ('er', 'u23', 800),
]

add_trades_specs = [get_fut_trade_spec(underlying, contract, notional=n) for underlying, contract, n in add_trades_list]
# add_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in add_trades_list]

add_trades_db = get_db_from_specs(add_trades_specs)

In [ ]:
add_trades, res = get_var_ts(add_trades_db)

In [ ]:
pd.concat([
    pd.concat(
        [
            (cob_ptf.nsmallest(1) / capital * 100),
            current_ptf.nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('daily').to_frame().T,
    pd.concat(
        [
            cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('weekly').to_frame().T
])

#### AA

In [ ]:
add_trades_list = [
    ('ff', 'q23', -1300),
    ('ff', 'z23', 1300),
    ('er', 'h24', 1650),
    ('er', 'h25', -1650),
]

add_trades_specs = [get_fut_trade_spec(underlying, contract, notional=n) for underlying, contract, n in add_trades_list]
# add_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in add_trades_list]

add_trades_db = get_db_from_specs(add_trades_specs)

In [ ]:
add_trades, res = get_var_ts(add_trades_db)

In [ ]:
pd.concat([
    pd.concat(
        [
            (cob_ptf.nsmallest(1) / capital * 100),
            current_ptf.nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('daily').to_frame().T,
    pd.concat(
        [
            cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('weekly').to_frame().T
])

In [ ]:
add_trades, res = get_var_ts(add_trades_db)

In [ ]:
pd.concat([
    pd.concat(
        [
            (cob_ptf.nsmallest(1) / capital * 100),
            current_ptf.nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('daily').to_frame().T,
    pd.concat(
        [
            cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('weekly').to_frame().T
])

In [ ]:
add_trades_list = [
    ('sfi', 'm24', -1000),
    ('sfr', 'm24', 1231),
]

add_trades_specs = [get_fut_trade_spec(underlying, contract, notional=n) for underlying, contract, n in add_trades_list]
# add_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in add_trades_list]

add_trades_db = get_db_from_specs(add_trades_specs)

In [ ]:
add_trades, res = get_var_ts(add_trades_db)

In [ ]:
pd.concat([
    pd.concat(
        [
            (cob_ptf.nsmallest(1) / capital * 100),
            current_ptf.nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('daily').to_frame().T,
    pd.concat(
        [
            cob_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            current_ptf.resample('W-FRI').sum().nsmallest(1) / capital * 100,
            pd.concat([cob_ptf, intraday_trades, add_trades], axis=1).sum(axis=1).resample('W-FRI').sum().nsmallest(1) / capital * 100
        ]
    ).set_axis(['cob', 'live', 'custom']).rename('weekly').to_frame().T
])

### ALL FUTURES

In [17]:
stirs = [
    'er',
    'sfr',
    'sfi',
    'ff'
]

govts = [
    'tu',
    'fv',
    'ty',
    'us',
    'du',
    'oe',
    'rx',
    'ub',
]

In [18]:
gen_futs = [
    *[f"{code}{i}" for code in stirs for i in range(2, 16+1)], 
    *[f"{code}{i}" for code in govts for i in range(1, 1+1)]
]

spec_futs = get_specific_future(gen_futs, bq=bq)

spec_fut_clarion = [fut.split(' ')[0] for fut in spec_futs]
spec_fut_clarion = [bbg_stir_to_clarion(fut) for fut in spec_fut_clarion]

In [19]:
spec_fut_clarion = [fut.split(' ')[0] for fut in spec_futs]
spec_fut_clarion = [bbg_stir_to_clarion(fut) for fut in spec_fut_clarion]

In [20]:
all_fut_list = [(fut[:-3].lower(), fut[-3:].lower()) for fut in spec_fut_clarion]
all_fut_specs = [get_fut_trade_spec(underlying, contract) for underlying, contract in all_fut_list]
all_fut_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in all_fut_list]

In [21]:
all_fut_db = get_db_from_specs(all_fut_specs)

In [22]:
all_fut_ts, res = get_var_ts(all_fut_db)

### MINIMIZE

In [ ]:
max_size = 10e3

In [ ]:
new_trades = all_fut_ts

In [ ]:
# x0 = rng.normal(size=new_trades.shape[1])
x0 = np.full(shape=new_trades.shape[1], fill_value=-100)
x0 = rng.normal(size=new_trades.shape[1]) * 1000

scales = [*np.hstack([10  ** np.arange(0, np.log10(max_size) + 1), - 10  ** np.arange(0, np.log10(max_size) + 1)])]
max_iter = 10
max_try = max_iter * len(scales)
k= 0 

In [ ]:
res_opt = None
for i in range(max_iter):
    for scale in scales:
        k = k + 1
        x0 = rng.normal(size=new_trades.shape[1]) * scale
        x0[x0 < -max_size] = -max_size
        x0[x0 > max_size] = max_size
        res_opt_ = minimize(
                lambda wgt: cvar_ptf(
                    current_ptf=current_ptf,
                    new_trades=new_trades,
                    wgt=wgt
                ),
                bounds=[(-max_size, max_size) for col in new_trades],
                x0=x0,
                # tol=0.001,
                method='Nelder-Mead', 
                # method='L-BFGS-B',
                # method='SLSQP',
                # constraints=[
                #     {
                #         'type': 'ineq',
                #         'fun': lambda wgt: 0.3 + var(df=d1, wgt=wgt, periods=522, diff=True, scale=-1)
                #     },
                #     {
                #         'type': 'ineq',
                #         'fun': lambda wgt: 1 + var(df=d1, wgt=wgt, periods=522, agg=True, diff=True, scale=-1)
                #     },
                # ],
                # options={
                #     'maxiter': 10000000000000,
                # }
            )
        if res_opt is None:
            res_opt = res_opt_
        
        if res_opt_['fun'] < res_opt['fun']:
            res_opt = res_opt_
        # clear_output()
        print(f"{k} - {max_try} {res_opt['x'].round()} {round(res_opt['fun'], 4)}", end='\r')

### MINIMIZE CVAR

In [ ]:
from scipy.optimize import linprog
from gioutils.utils import filter_high_corr

In [ ]:
new_trades = filter_high_corr(all_fut_ts, all_fut_ts.corrwith(current_ptf).abs(), threshold=.98)

In [ ]:
new_trades

In [ ]:
from scipy.special import comb, perm

In [ ]:
max_size = 10e3
max_iter = int(1e3)
min_ = 1e9
n_trades = 5
total = comb(new_trades.shape[1], n_trades, exact=True)
for i, comb_ in enumerate(combinations(new_trades.columns, min(n_trades, new_trades.shape[1]) if n_trades > 0 else new_trades.shape[1])):
    new_trades_ = new_trades.loc[:, comb_]
    r, wgt = min_cvar_linprog(
        current_ptf=current_ptf, 
        new_trades=new_trades_,
        min_size=-max_size,
        max_size=max_size
    )
    cvar_ = cvar_ptf(current_ptf=current_ptf, new_trades=new_trades_, wgt=wgt)
    var_ = var_ptf(current_ptf=current_ptf, new_trades=new_trades_, wgt=wgt)
    
    if var_ < min_:
        min_ = var_
        wgt_ = wgt
        ress = new_trades_, r, wgt, cvar_, var_
        
    print(f"{i + 1}/{total} - {ress[-3].round()} {round(ress[-2], 4)} {round(ress[-1], 4)}", end='\r')


In [ ]:
res_opt = ress[1]
new_trades_ = ress[0]

In [ ]:
new_trades_clarion = [get_clarion_fut_trade_spec(clarion, underlying, contract) for underlying, contract in [(fut[:-3], fut[-3:]) for fut in new_trades_]]
deltas = runner.run(metrics=[clarion_metrics['delta']], data_list=new_trades_clarion).loc[:, ['id', 'delta']]
clear_output()

In [ ]:
var_close = - cob_ptf.quantile(0.05, interpolation='weibull') / flag
var_intraday = - current_ptf.quantile(0.05, interpolation='weibull') / flag

trades = pd.Series(wgt_.round(0), index=new_trades_.columns.str.upper(), name='quantity')
var_new = var_ptf(current_ptf=current_ptf, new_trades=new_trades_, wgt=wgt_)
deltas_opt = pd.Series(deltas['delta'].to_numpy(), index=deltas['id'].str.upper())
delta_usd = (trades * deltas_opt).rename('delta$')

trades_show = pd.concat([trades, delta_usd], axis=1)
sum_ = trades_show.sum().rename('TOTAL')
sum_.iloc[0] = np.nan
sum_= sum_.to_frame().T
trades_show = pd.concat([trades_show, sum_])

In [ ]:
display(w.HTML(f"<h3>Close VaR:{'&nbsp;'*40}{var_close: .2%}</h3>"))
display(w.HTML(f"<h3>VaR with Intraday (below): {'&nbsp;'*11}{var_intraday: .2%}</h3>"))
display(intraday_rates)
display(w.HTML(f"<h3>VaR after the below trades:{'&nbsp;'*11}{var_new: .2%}</h3>"))
display(trades_show)

In [ ]:
display(w.HTML(f"<h3>Close VaR:{'&nbsp;'*40}{var_close: .2%}</h3>"))
display(w.HTML(f"<h3>VaR with Intraday (below): {'&nbsp;'*11}{var_intraday: .2%}</h3>"))
display(intraday_rates)
display(w.HTML(f"<h3>VaR after the below trades:{'&nbsp;'*11}{var_new: .2%}</h3>"))
display(trades_show)

In [ ]:
send_live_var_email(
    var_close, 
    var_intraday, 
    var_new, 
    intraday_rates, 
    trades_show,
    test=True
)

### TAIL

In [23]:
n = 45
try_trades = all_fut_ts.iloc[:, :n]

In [28]:
max_size = 2000
n_wgt = 12
k = 2
wgt_grid = np.linspace(-max_size, max_size, n_wgt).astype(int)
wgt_grid

array([-2000, -1636, -1272,  -909,  -545,  -181,   181,   545,   909,
        1272,  1636,  2000])

In [38]:
i = 0
min_d_tail = np.inf
min_w_tail = np.inf
mins = []
max_save = 100

tot = int(comb(try_trades.shape[1], k) * perm(len(wgt_grid), k))
for comb_ in combinations(try_trades.columns, k):
    for perm_ in permutations(wgt_grid, k):
        i += 1
        # print(f"{i} / {tot}", end='\r')
        new_trades = try_trades.loc[:, comb_] * perm_
        new_trades_w = new_trades.resample('W-FRI').sum()
        d_tail = - dv.tail(df=pd.concat([current_ptf, new_trades], axis=1).dropna(), capital=capital, add=False).squeeze()
        w_tail = - wv.tail(df=pd.concat([current_ptf_w, new_trades_w], axis=1).dropna(), capital=capital, add=False).squeeze()
        
        if len(mins) < 100:
            mins.append((comb_, perm_, d_tail, w_tail))
            mins.sort(key=lambda lst: lst[2], reverse=True)
        
        elif d_tail < min_d_tail:
            mins.append((comb_, perm_, d_tail, w_tail))
            mins.pop(0)
        min_d_tail = min(min_d_tail, d_tail)
        min_w_tail = min(min_w_tail, w_tail)
        print(f"{i} / {tot}, d_tail:  {mins[-1][0]} {mins[-1][1]} d_tail:{round(mins[-1][2], 2)} w_tail:{round(mins[-1][3], 2)}", end='\r')
        # break
    # break

KeyboardInterrupt: 

In [40]:
dv.tail(pd.concat([current_ptf, all_fut_ts['sfru23'] * -3500], axis=1).dropna(), capital=capital)

2023-03-14   -1.6390
Freq: B, dtype: float64

In [49]:
wv.tail(pd.concat([current_ptf_w, (all_fut_ts['sfru23'] * -3500).resample('W-FRI').sum()], axis=1).dropna(), capital=capital)

2023-03-17   -2.8572
Freq: W-FRI, dtype: float64

In [27]:
res = pd.DataFrame(mins, columns=['contracts', 'quantities', 'daily_tail', 'weekly_tail'])
res = res.assign(average_tail=res.iloc[:, -2:].mean(axis=1)).sort_values('average_tail')
res.head(10)

contracts          quantities  daily_tail  weekly_tail  \
95  (sfrm23, sfru23)  (-2000.0, -1636.0)      1.9610       2.0208   
96  (sfrm23, sfru23)  (-1636.0, -2000.0)      1.8699       2.1930   
97  (sfrm23, sfrz23)  (-1636.0, -2000.0)      1.8270       2.4144   
93   (eru23, sfru23)  (-1636.0, -2000.0)      2.0636       2.4920   
92   (eru23, sfru23)  (-2000.0, -1636.0)      2.1978       2.3863   
98  (sfru23, sfrz23)  (-2000.0, -1636.0)      1.4803       3.1480   
99  (sfru23, sfrz23)  (-1636.0, -2000.0)      1.4477       3.1883   
91   (eru23, sfrm23)  (-1636.0, -2000.0)      2.5641       2.1072   
90   (eru23, sfrm23)  (-2000.0, -1636.0)      2.6072       2.0980   
94   (eru23, sfrz23)  (-1636.0, -2000.0)      2.0206       2.7134   

    average_tail  
95        1.9909  
96        2.0315  
97        2.1207  
93        2.2778  
92        2.2921  
98        2.3141  
99        2.3180  
91        2.3357  
90        2.3526  
94        2.3670

In [ ]:
nc = -1000

In [ ]:
dv.tail(capital=capital)

In [ ]:
dv.tail((nc * try_trades['sfrz23']), add=True, capital=capital)

In [ ]:
wv.tail((nc * try_trades['sfrz23']).resample('W-FRI').sum(), add=True, capital=capital)

In [ ]:
ress = {}
save_max = 1000
max_iter = 1e5
i = 0
min1 = 1e9
while True:
    i += 1
    wgt = rng.choice(wgt_grid, new_trades.shape[1]) if not isinstance(wgt_grid, dict) else np.asanyarray([rng.choice(wgt_grid[col])  for col in new_trades])
    var_ = var_ptf(current_ptf=current_ptf, new_trades=new_trades, wgt=wgt)
    ress[tuple(wgt)] = var_
    
    if len(ress) > save_max:
        ress.pop(max(ress, key=ress.get))
    
    min2 = min(ress.values())
#     if i == 200:
#         break
    if min2 < min1:
        min1 = min2
        wgt_ = wgt
        
    print(f"{i} - {wgt_} {round(min1, 4)}", end='\r')
    if i + 1 == max_iter:
        break

In [ ]:
res = pd.Series(ress, name='quantity').rename_axis(new_trades.columns).sort_values()

In [ ]:
var_close = - dv.var() / flag

var_intraday = - pd.concat([dv.hist.sum(axis=1).rename('ptf'), ts], axis=1).sum(axis=1).quantile(0.05, interpolation='weibull') / flag
trades = pd.Series(res.reset_index().iloc[0, :-1].to_numpy(), index=res.reset_index().columns[:-1].str.upper(), name='quantity').to_frame()
var_new = res.reset_index().iloc[0, -1]

In [ ]:
display(w.HTML(f"<h3>Close VaR:{'&nbsp;'*40}{var_close: .2%}</h3>"))
display(w.HTML(f"<h3>VaR with Intraday (below): {'&nbsp;'*11}{var_intraday: .2%}</h3>"))
# display(db)
display(w.HTML(f"<h3>VaR after the below trades:{'&nbsp;'*11}{var_new: .2%}</h3>"))
display(trades)

In [ ]:
def get_clarion_fut_trade_spec(underlying, contract):
    instrument = 'lfut' if underlying.lower() in lfut_list else 'sfut'
    gen = fut_gen_map.get(underlying.lower())

    return {
        'instrument': instrument,
        'ccy': gen_ccy_map.get(gen),
        'gen': gen,
        'mty_tnr': contract,
        'pay_type': 'pay',
        'notional': 1,
    }

In [ ]:
db_ = get_db_from_clarion_positions(df.query('Category == "rates"'))

In [ ]:
a = bh.bhTsFiRemoteRiskCalc(
    tradesdb=db_,
    startdate=today() - BDay() - parse_offset('2y'),
    enddate=today() - BDay(),
    pblevals=['PV'],
    irbtdbname=SQL.FITradePricer.db,
    irbtdbsrvname=SQL.FITradePricer.server,
    remoteservername=SQL.IRBTServer.server,
    remoteserverport=SQL.IRBTServer.port,
    timeout=120
)

In [ ]:
db_

In [ ]:
a.py.query("Status=='OK' and Metric == 'Rate'")

In [ ]:
new_trades.diff() 

In [ ]:
new_trades / new_trades.iloc[0]